In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

##### Definições de parâmetros

In [2]:
INPUT_SIZE = 784        # Número de pixeis na imagem (28 x 28)
OUTPUT_SIZE = 10        # Número de classes (0, 1, 2, ...9) 

#### Visualizando a vetorização da entrada
![SegmentLocal](Imagens/mnist_pytorch_3b1b_28x28.gif "segment")

##### Definições de Hiperpârametros

In [3]:
LEARNING_RATE = 0.005   # Taxa de aprendizado da rede
EPOCHS = 5              # Quantos vezes a rede passa pelo conjunto de dados
BATCH_SIZE = 64         # Número de imagens passados para rede de uma só vez
HIDDEN_SIZE = 64        # Número de neurônios nas camadas

##### Definindo a classe da rede neural

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):   # Definindo as camadas da rede
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)       
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):                                       # Definindo o foward propagation
        x = torch.flatten(x, start_dim=1)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        return x

#### Visualizando a rede neural
![SegmentLocal](Imagens/mnist_pytorch_3b1b.gif "segment")

#### Datasets

In [5]:
mnist_train = MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())      # Imagens de treino (60000)
mnist_test = MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())      # Imagens de teste  (10000)

1.7%

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



102.8%

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw




6.2%

100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



112.7%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



#### Inicializando a rede

In [6]:
model = NeuralNetwork(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), LEARNING_RATE)

### Loop de treino

In [7]:
mnist_dataloader = DataLoader(mnist_train, batch_size=BATCH_SIZE, drop_last=True)

for epoch in range(EPOCHS):
    total_loss = 0
    for image, labels in mnist_dataloader:
        prediction = model(image)
        loss = criterion(prediction, labels)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1} loss: {total_loss / (len(mnist_train) / BATCH_SIZE)}")

Epoch 1 loss: 0.27255119517644244
Epoch 2 loss: 0.1327110718590518
Epoch 3 loss: 0.1009925508412222
Epoch 4 loss: 0.08511960146497004
Epoch 5 loss: 0.07694078705984478


#### Loop de testes

In [8]:
mnist_dataloader = DataLoader(mnist_test, batch_size=1, drop_last=True)

with torch.no_grad():
    total_correct = 0
    for image, label in mnist_dataloader:
        prediction = model(image)
        index = torch.argmax(prediction)
        if index == label.item():
            total_correct += 1
    print(f"Percentage of correct answers: {((total_correct*100)/len(mnist_test)):.2f}%")


Percentage of correct answers: 97.26%
